```{contents}
```
## **Token Budgeting in LangGraph**

**Token budgeting** is the discipline of **controlling, allocating, monitoring, and optimizing token usage** across an LLM workflow so that the system remains **cost-efficient, latency-bounded, stable, and scalable**.

In LangGraph, token budgeting is a **first-class production concern** because graphs execute **multi-step, cyclic, multi-agent workflows** where uncontrolled token growth can cause **cost explosions, slowdowns, and failures**.

---

### **1. Why Token Budgeting Is Critical in LangGraph**

Unlike simple LLM calls, LangGraph workflows involve:

* Loops (ReAct, reflection, self-healing)
* Multi-agent collaboration
* Long-running sessions
* Persistent memory

Without token control, these cause:

| Failure Mode             | Effect                  |
| ------------------------ | ----------------------- |
| Unbounded context growth | Context window overflow |
| High latency             | Slow responses          |
| Cost explosion           | Budget violations       |
| Agent instability        | Hallucinations & drift  |
| Production outages       | Failed executions       |

---

### **2. Token Budgeting Objectives**

| Objective            | Description                 |
| -------------------- | --------------------------- |
| Context safety       | Stay within model limits    |
| Cost predictability  | Bound spending per run      |
| Latency control      | Reduce prompt size          |
| Quality preservation | Keep essential information  |
| Scalability          | Serve many concurrent users |

---

### **3. Token Budget Architecture in LangGraph**

```
User Input
   ↓
State Memory (messages, tools, plans)
   ↓
Token Controller
   ├─ Budget Allocation
   ├─ Trimming Policy
   ├─ Compression Policy
   ├─ Routing Policy
   └─ Fallback Policy
   ↓
LLM Node Execution
```

Token management occurs at **every LLM node invocation**.

---

### **4. Where Tokens Accumulate**

| Source                      | Tokens   |
| --------------------------- | -------- |
| Conversation history        | High     |
| Tool outputs                | Medium   |
| Agent messages              | High     |
| Plans / reflections         | Medium   |
| Long-term memory retrievals | Variable |
| System instructions         | Constant |

LangGraph stores most of these inside the **shared state**.

---

### **5. Core Token Control Techniques**

#### **A. Hard Token Limits**

```python
llm = ChatOpenAI(max_tokens=512)
```

Controls output length.

---

#### **B. State Trimming Policy**

```python
def trim_messages(state, max_tokens=2000):
    while count_tokens(state["messages"]) > max_tokens:
        state["messages"].pop(0)
```

Applied before LLM node execution.

---

#### **C. Sliding Window Memory**

Keep only most recent context:

```
[System] + [Last N messages] + [Current task]
```

---

#### **D. Semantic Compression**

Summarize older context:

```python
def compress(state):
    summary = summarize(state["messages"])
    state["messages"] = [summary]
```

---

#### **E. Budget Allocation Per Node**

| Node         | Token Budget |
| ------------ | ------------ |
| Planner      | 1000         |
| Executor     | 800          |
| Reflection   | 500          |
| Tool Summary | 300          |

---

### **6. Token-Aware Router Example**

```python
def route(state):
    if count_tokens(state["messages"]) > 3000:
        return "compress"
    return "reason"
```

---

### **7. Token Budgeting in Cyclic Graphs**

In loops, tokens accumulate rapidly.
LangGraph requires **loop-level token guards**:

```python
config = {"recursion_limit": 10}
```

And per-cycle trimming:

```python
def loop_guard(state):
    state["messages"] = trim_to_budget(state["messages"], 2000)
```

---

### **8. Production Token Budgeting Strategy**

| Layer    | Strategy                      |
| -------- | ----------------------------- |
| LLM Call | Hard max_tokens               |
| State    | Sliding window + compression  |
| Loop     | Token guard + recursion limit |
| Agent    | Role-based token quotas       |
| System   | Cost monitoring               |
| Fallback | Smaller model routing         |

---

### **9. Token Budgeting Variants**

| Variant          | Use Case              |
| ---------------- | --------------------- |
| Strict Budget    | Finance / enterprise  |
| Soft Budget      | Creative apps         |
| Adaptive Budget  | Load-aware systems    |
| Per-Agent Budget | Multi-agent platforms |
| Session Budget   | SaaS cost control     |

---

### **10. Monitoring & Enforcement**

| Metric         | Purpose           |
| -------------- | ----------------- |
| Tokens / run   | Cost              |
| Tokens / user  | Quotas            |
| Tokens / agent | Optimization      |
| Tokens / node  | Hotspot detection |
| Latency        | Performance       |

---

### **11. Mental Model**

> **Tokens are the currency of LLM systems.
> LangGraph is the central bank.**

Every node spends from the shared token economy.


### Demonstration

In [4]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List
from langchain_openai import ChatOpenAI
from langchain_classic.schema import BaseMessage, HumanMessage

# ----------------------------
# State
# ----------------------------

class State(TypedDict):
    messages: List[BaseMessage]
    steps: int

# ----------------------------
# Utilities
# ----------------------------

def count_tokens(messages):
    return sum(len(m.content.split()) for m in messages)

def trim_to_budget(messages, budget=120):
    while count_tokens(messages) > budget:
        messages.pop(0)
    return messages

# ----------------------------
# LLM
# ----------------------------

llm = ChatOpenAI(model="gpt-4o-mini", max_tokens=60)

# ----------------------------
# Nodes
# ----------------------------

def reason(state: State):
    state["messages"] = trim_to_budget(state["messages"], 120)
    reply = llm.invoke(state["messages"])
    state["messages"].append(reply)
    state["steps"] += 1
    return state

def compress(state: State):
    summary = llm.invoke([HumanMessage(content=f"Summarize briefly: {state['messages']}")])
    state["messages"] = [summary]
    return state

def router(state: State):
    if state["steps"] >= 3:
        return END
    if count_tokens(state["messages"]) > 100:
        return "compress"
    return "reason"

# ----------------------------
# Graph
# ----------------------------

builder = StateGraph(State)

builder.add_node("reason", reason)
builder.add_node("compress", compress)

builder.set_entry_point("reason")

builder.add_conditional_edges("reason", router, {
    "compress": "compress",
    "reason": "reason",
    END: END
})

builder.add_edge("compress", "reason")

graph = builder.compile()

# ----------------------------
# Run
# ----------------------------

initial_state = {
    "messages": [HumanMessage(content="Explain LangGraph token budgeting with examples.")],
    "steps": 0
}

result = graph.invoke(initial_state, config={"recursion_limit": 10})

print("Final Steps:", result["steps"])
print("Final Token Count:", count_tokens(result["messages"]))
print("\nFinal Messages:\n")

for m in result["messages"]:
    print(m.type.upper(), ":", m.content[:200])


Final Steps: 3
Final Token Count: 96

Final Messages:

AI : LangGraph token budgeting involves managing the allocation of tokens consumed by large language models (LLMs) to enhance performance, cost-efficiency, and overall effectiveness in various applications
AI : allocated optimally to maintain high-quality outputs while minimizing costs. Here are some key aspects to consider for token budgeting in LangGraph or similar scenarios:

1. **Understanding Token Limi
